In [4]:
with open("shakespeare.txt","r",encoding="utf-8") as f:
    raw_text = f.read()
print(len(raw_text))
print(raw_text[:99])

6446
 From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as 


In [5]:
import re

In [6]:
preprocessed=re.split(r'([,.:;?_!"()\']|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:99])

1321
['From', 'fairest', 'creatures', 'we', 'desire', 'increase', ',', 'That', 'thereby', 'beauty', "'", 's', 'rose', 'might', 'never', 'die', ',', 'But', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', ',', 'His', 'tender', 'heir', 'might', 'bear', 'his', 'memory', ':', 'But', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', ',', 'Feed', "'", 'st', 'thy', 'light', "'", 's', 'flame', 'with', 'self-substantial', 'fuel', ',', 'Making', 'a', 'famine', 'where', 'abundance', 'lies', ',', 'Thy', 'self', 'thy', 'foe', ',', 'to', 'thy', 'sweet', 'self', 'too', 'cruel', ':', 'Thou', 'that', 'art', 'now', 'the', 'world', "'", 's', 'fresh', 'ornament', ',', 'And', 'only', 'herald', 'to', 'the', 'gaudy', 'spring', ',', 'Within', 'thine', 'own', 'bud', 'buriest', 'thy']


In [7]:
words=sorted(set(preprocessed))
print(len(words))

544


In [8]:
vocab={token:i for i,token in enumerate(words)}
for i,j in enumerate(vocab.items()):
  print(j)
  if i>9:
    break

("'", 0)
('(', 1)
(')', 2)
(',', 3)
('.', 4)
(':', 5)
(';', 6)
('?', 7)
('A', 8)
('Ah', 9)
('And', 10)


In [9]:
words.extend(["<|unk|>"])
vocab={token:i for i,token in enumerate(words)}
print(len(words))

545


In [10]:
class Tokenizer:
  def __init__(self, vocab):
    self.str_to_int = vocab
    self.int_to_str = { i:s for s,i in vocab.items()}
  def encode(self, text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
    token_ids=[self.str_to_int[s] for s in preprocessed]
    return token_ids
  def decode(self, token_ids):
    tokens=[self.int_to_str[i] for i in token_ids]
    return tokens

In [11]:
text='Hello how are thee my lad'

In [12]:
tokenizer=Tokenizer(vocab)
encode=tokenizer.encode(text)
print(encode)

[544, 268, 102, 458, 336, 544]


In [13]:
decode=tokenizer.decode(encode)
print(decode)

['<|unk|>', 'how', 'are', 'thee', 'my', '<|unk|>']


In [15]:
# BPE
from collections import Counter
class BPE:
  def __init__(self,corpus,vocab_size,max_iter=None):
    self.corpus=corpus
    self.vocab_size=vocab_size
    self.word_freq=Counter()
    self.splits={}
    self.merges={}
    self.max_iter=max_iter


In [16]:
def train(self):
  for document in self.corpus:
    words=document.split()
    self.word_freq+=Counter(words)

  for word in self.word_freq:
    self.splits[word]=list(word)+['<\w>']

In [17]:
def get_pairs_freq(self):
  pairs_freq=Counter()
  for word,freq in self.word_freq.items():
    split=self.splits[word]
    for i in range(len(split)-1):
      pairs_freq[(split[i],split[i+1])]+=freq
  return pairs_freq

In [19]:
  while len(self.merges)<self.vocab_size:
    pair_freq=self.get_pairs_freq()
    if not pair_freq:
      break
    pair=max(pair_freq,key=pair_freq.get)
    self.update_splits(pair)
    self.merges[pair]=pair+pair


NameError: name 'self' is not defined

In [20]:
def update_splits(self, pair):
    lhs, rhs = pair
    for word in list(self.splits.keys()):
        new_split = []
        cursor = 0
        while cursor < len(self.splits[word]):
            if cursor + 1 < len(self.splits[word]) and \
               self.splits[word][cursor] == lhs and \
               self.splits[word][cursor + 1] == rhs:
                new_split.append(lhs + rhs)
                cursor += 2
            else:
                new_split.append(self.splits[word][cursor])
                cursor += 1
        self.splits[word] = new_split

In [21]:
def tokenize(self, text):
    splits = [list(word) + ['</w>'] for word in text.split()]
    for lhs, rhs in self.merges:
        for idx, split in enumerate(splits):
            new_split = []
            cursor = 0
            while cursor < len(split):
                if cursor + 1 < len(split) and split[cursor] == lhs and split[cursor + 1] == rhs:
                    new_split.append(lhs + rhs)
                    cursor += 2
                else:
                    new_split.append(split[cursor])
                    cursor += 1
            splits[idx] = new_split
    return sum(splits, [])

In [23]:
from collections import Counter
class BPE:
  def __init__(self,corpus,vocab_size,max_iter=None):
    self.corpus=corpus
    self.vocab_size=vocab_size
    self.word_freq=Counter()
    self.splits={}
    self.merges={}
    self.max_iter=max_iter

  def train(self):
    for document in self.corpus:
      words=document.split()
      self.word_freq+=Counter(words)

    for word in self.word_freq:
      self.splits[word]=list(word)+['<\w>']


  def get_pairs_freq(self):
    pairs_freq=Counter()
    for word,freq in self.word_freq.items():
      split=self.splits[word]
      for i in range(len(split)-1):
        pairs_freq[(split[i],split[i+1])]+=freq
    return pairs_freq

    while len(self.merges)<self.vocab_size:
      pair_freq=self.get_pairs_freq()
      if not pair_freq:
        break
      pair=max(pair_freq,key=pair_freq.get)
      self.update_splits(pair)
      self.merges[pair]=pair+pair

  def update_splits(self, pair):
    lhs, rhs = pair
    for word in list(self.splits.keys()):
        new_split = []
        cursor = 0
        while cursor < len(self.splits[word]):
            if cursor + 1 < len(self.splits[word]) and \
               self.splits[word][cursor] == lhs and \
               self.splits[word][cursor + 1] == rhs:
                new_split.append(lhs + rhs)
                cursor += 2
            else:
                new_split.append(self.splits[word][cursor])
                cursor += 1
        self.splits[word] = new_split

  def tokenize(self, text):
    splits = [list(word) + ['</w>'] for word in text.split()]
    for lhs, rhs in self.merges:
        for idx, split in enumerate(splits):
            new_split = []
            cursor = 0
            while cursor < len(split):
                if cursor + 1 < len(split) and split[cursor] == lhs and split[cursor + 1] == rhs:
                    new_split.append(lhs + rhs)
                    cursor += 2
                else:
                    new_split.append(split[cursor])
                    cursor += 1
            splits[idx] = new_split
    return sum(splits, [])




In [24]:
corpus = ["highest", "higher", "lower", "lowest", "cooler", "coolest"]
bpe = BPE(corpus, vocab_size=17)
bpe.train()

sample_text = "highest higher lower lowest cooler coolest"
tokens = bpe.tokenize(sample_text)
print(tokens)

['h', 'i', 'g', 'h', 'e', 's', 't', '</w>', 'h', 'i', 'g', 'h', 'e', 'r', '</w>', 'l', 'o', 'w', 'e', 'r', '</w>', 'l', 'o', 'w', 'e', 's', 't', '</w>', 'c', 'o', 'o', 'l', 'e', 'r', '</w>', 'c', 'o', 'o', 'l', 'e', 's', 't', '</w>']
